### Notes
(filter only for commerical (non-housing center))
(throw warnings when radii is too big) (100 to 200m)
(sort aid by activity descending)


lid -> [pid] (flip) groupby lid, set(pid)

aid' -> [lid] (generate groupings)

aid' -> [pid] (expand groupings)
aid -> [pid] (sort)
pid -> aid (revert)


Plan:
Housing IDs treated seperately.

1. 


### New Plan:


In [1]:
#%%

%load_ext autoreload
%autoreload 2

import random
import pandas as pd
from typing import Dict, List, Tuple, Set
from src.problem import *
from src.round import *
from src.heuristics import *
from src.utils import *
from src.config import *
import folium
import numpy as np
from folium.plugins import FastMarkerCluster, MarkerCluster

# Data loading and preprocessing
RAW_DATA_DIR = PROJECT_ROOT / 'data' / 'raw'

# Need to preprocess the adult_df
# act_df?
df_act = pd.read_csv(RAW_DATA_DIR / "usa_va_charlottesville_city_adult_activity_location_assignment_week.csv")

# aloc_df?
# rloc_df?
df_aloc = pd.read_csv(RAW_DATA_DIR / "usa_va_charlottesville_city_activity_locations.csv")
df_rloc = pd.read_csv(RAW_DATA_DIR / "usa_va_charlottesville_city_residence_locations.csv")

# Define lid
df_aloc['lid'] = df_aloc['alid']
df_rloc['lid'] = df_rloc['rlid'] + HOME_SHIFT

# Location dataframe (with both activity and residential locations)
df_loc = pd.concat([df_aloc[['lid', 'longitude', 'latitude']], df_rloc[['lid', 'longitude', 'latitude']]]).set_index('lid')

mat_aloc = df_aloc[['longitude', 'latitude']].to_numpy()



/home/ubuntu/anaconda3/envs/facility/lib/python3.9/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


In [2]:
# Shift df_act's rloc by HOME_SHIFT

# All coordinates of residential areas
coord_r = set(df_rloc[['latitude', 'longitude']].apply(tuple, axis=1).tolist())

df_act['coord'] = df_act[['latitude', 'longitude']].apply(lambda x: (x.latitude, x.longitude), axis = 1)
df_act.loc[df_act.coord.isin(coord_r), 'lid'] += HOME_SHIFT

In [29]:
df_act.head()

,hid,pid,activity_number,activity_type,start_time,duration,lid,longitude,latitude,travel_mode,coord
0,2208253,5586585,0,1,0,27900,1001018209,-78.488467,38.043025,-1,"(38.0430255, -78.4884675)"
1,2208253,5586585,2,2,28800,17100,82246,-78.414920,38.157530,-1,"(38.1575300000001, -78.4149199999999)"
2,2208253,5586585,4,4,46800,1200,86726,-78.498790,38.139160,-1,"(38.1391600000001, -78.49879)"
3,2208253,5586585,6,2,48600,15300,82246,-78.414920,38.157530,-1,"(38.1575300000001, -78.4149199999999)"
4,2208253,5586585,8,1,64800,36000,1001018209,-78.488467,38.043025,-1,"(38.0430255, -78.4884675)"


In [32]:
# Assert that activity locations are a subset of total locations
set(df_act['lid']) - set(df_loc.index) == set()

True

In [33]:
# Generate single-linkage clusters (aid' -> lid)
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import BallTree
from networkx.utils.union_find import UnionFind

THRESHOLD = .025 # distance in km
EARTH_RADIUS = 6371 # kilometers


def generate_groups(loc_mat, threshold):  
    '''
    loc_mat: a N x 2 matrix holding spherical coordinates
    threshold: a threshold in km for linkage clustering
    '''
    tree = BallTree(loc_mat, leaf_size=40, metric='haversine')
    # Threshold all values less than radius r
    
    threshold_radius = threshold / EARTH_RADIUS
    groups = tree.query_radius(loc_mat, r=threshold_radius)
    valid_groups = [group for group in groups if len(group) > 1]

    uf = UnionFind(range(len(loc_mat)))
    for group in groups:
        uf.union(*group)
    agg_groups = list(uf.to_sets())
    return agg_groups

# Generates aid' -> activity_index (aids are tentative)
groups = generate_groups(mat_aloc, THRESHOLD)
aid2lid = [[df_aloc.iloc[e]['lid'] for e in g] for g in groups]

In [34]:
# For each location, get a list of different pids associated with it
# Some locations may not appear ...
lid2pid = df_act.groupby("lid")["pid"].agg(set).to_dict()


In [38]:
aid_raw = [None] * len(aid2lid)

for i, lids in enumerate(aid2lid):
    members = set().union(*[lid2pid.get(i, []) for i in lids])
    activity = len(members)
    aid_raw[i] = {
        'members': members,
        'locations': lids,
        'activity': activity,
    }
    # Check if there are duplicate members in aggregated markers
    # YES, there are members that are duplicate across aggregated markers
#     if len(members) != sum([len(lid2pid.pid.get(i, [])) for i in lids]):
#         print(i, lids, [lid2pid.pid.get(i, []) for i in lids])
print(len(aid_raw))


6957


In [39]:
aid2pid = [{'aid': i, **group} for i, group in enumerate(sorted(aid_raw, key=(lambda x: (-x['activity'], min(members)))))]

In [ ]:

# Visualization of different groups
# Objective: show different markers, use color of marker to denote different aggregate groups?

